In [1]:
import spacy
import pathlib
import pandas as pd
from memo import grid, memlist

In [2]:
data = [] 

@memlist(data=data)
def run_experiment(mod, datafile, cap=False):
    # load in spacy model 
    nlp = spacy.load(mod)
    
    # load in names
    names = pathlib.Path(datafile).read_text().split("\n")
    
    # try to detect the names using simple "je m'appelle"
    ent_types = [d[-1].ent_type_ for d in nlp.pipe(f"Bonjour je m'appelle {n.capitalize() if cap else n}" for n in names)]
    results_df = pd.DataFrame({"name": names, "ent_type": ent_types}).groupby("ent_type").count()
    return dict(results_df['name'])

for setting in grid(datafile=['arabic-names.txt', 'french-names.txt'],
                    mod=['fr_core_news_sm', 'fr_core_news_md'],
                    cap=[True, False]):
    run_experiment(**setting)

Output()

In [26]:
def clean_data(dataf):
    return (dataf
            .assign(NO_ENT=lambda d: d[''])
            .drop(columns=[''])
            .fillna(0)
            .assign(total=lambda d: d[['LOC', 'MISC', 'ORG', 'PER', 'NO_ENT']].sum(axis=1))
            .assign(p_person=lambda d: d['PER']/d['total'])
            .assign(p_none=lambda d: d['NO_ENT']/d['total']))

# pd.DataFrame(data).assign(NO_ENT=lambda d: d['']).drop(columns=['']).fillna(0)

In [27]:
pd.DataFrame(data).pipe(clean_data)

,datafile,mod,cap,LOC,MISC,ORG,PER,NO_ENT,total,p_person,p_none
0,arabic-names.txt,fr_core_news_sm,True,327.0,37.0,33.0,355.0,15,767.0,0.462842,0.019557
1,arabic-names.txt,fr_core_news_sm,False,0.0,0.0,0.0,0.0,767,767.0,0.000000,1.000000
2,arabic-names.txt,fr_core_news_md,True,178.0,130.0,19.0,417.0,23,767.0,0.543677,0.029987
3,arabic-names.txt,fr_core_news_md,False,0.0,0.0,0.0,0.0,767,767.0,0.000000,1.000000
4,french-names.txt,fr_core_news_sm,True,423.0,51.0,16.0,507.0,3,1000.0,0.507000,0.003000
5,french-names.txt,fr_core_news_sm,False,424.0,51.0,16.0,508.0,1,1000.0,0.508000,0.001000
6,french-names.txt,fr_core_news_md,True,149.0,293.0,10.0,535.0,13,1000.0,0.535000,0.013000
7,french-names.txt,fr_core_news_md,False,149.0,294.0,10.0,536.0,11,1000.0,0.536000,0.011000
